In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.serialize import model_to_json, model_from_json
import pandas as pd
import json
from datetime import timedelta
import numpy as np
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s :: %(levelname)s :: %(message)s")
from glob import glob
from tqdm import tqdm
import os

In [ ]:
df_grouped_by_week = pd.read_parquet('./datasets/curated/almacenes_si_curated_by_week.parquet')

In [ ]:
# saco archivo de descuentos que tiene la info de descuentos por familia, por semana 
discount_campaigns = df_grouped_by_week[df_grouped_by_week['date_week'].between(pd.to_datetime('2023-01-01'),pd.to_datetime('2023-12-31'))]
discount_campaigns['familia'] = discount_campaigns['combination'].apply(lambda x: x[:3])
discount_campaigns = discount_campaigns[['familia','date_week','discount_for_event', 'campaign']]
discount_campaigns = discount_campaigns.groupby(['date_week'],as_index=False)['discount_for_event'].max()
discount_campaigns['campaign'] = discount_campaigns['discount_for_event'].apply(lambda x: 1 if x != 0.0 else 0)
discount_campaigns = discount_campaigns[['date_week', 'discount_for_event', 'campaign']]
discount_campaigns.to_csv('./archivos_insumo/archivo_insumo_campañas_2023.csv', index = False)
discount_campaigns.head()

In [ ]:
# Generar fechas semanales para el año 2024, cada lunes
dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='W-MON')

# Generar valores aleatorios entre 0 y 70
discount_for_event = np.random.randint(0, 71, size=len(dates))

# Crear el DataFrame
discount_campaigns = pd.DataFrame({'ds': dates, 'discount_for_event': discount_for_event})
discount_campaigns['campaign'] = discount_campaigns['discount_for_event'].apply(lambda x: 1 if x != 0.0 else 0)


In [77]:
def load_model_from_json(file_path):
    with open(file_path, "rb") as file:
        prophet_model = model_from_json(file.read())
        
    return prophet_model

def make_predictions(model, future_regressors, year):
    # Obtener la última fecha histórica
    last_date = max(model.history_dates) + timedelta(weeks=1)
    
    # Crear un DataFrame con las fechas futuras hasta el año especificado, por semana comenzando en lunes
    future_dates = pd.date_range(start=last_date, end=f'{year}-12-31', freq='W-MON')
    
    # Crear el DataFrame con las fechas y los regresores
    future = pd.DataFrame({'ds': future_dates})
    
    # Añadir los regresores al DataFrame futuro
    future = future.merge(future_regressors, how='left', on='ds')
    
    # Llenar NaNs con 0 si es necesario (asegúrate que esto tenga sentido para tu caso)
    future.fillna(0, inplace=True)
    
    # Hacer la predicción
    forecast = model.predict(future)
    
    return forecast



In [78]:
# Cargar el modelo desde el archivo JSON
model = load_model_from_json('./serialized_models/201AA3.json')

# Hacer la predicción hasta el año 2025
# future_regressors = discount_campaigns
# forecast, future = make_predictions(model, future_regressors, 2024)
# forecast[['ds', 'yhat']]

In [81]:
model.history['ds'].max()

Timestamp('2023-12-25 00:00:00')

In [ ]:
discount_campaigns.to_csv('./archivos_insumo/archivo_insumo_dummy_campañas_2024.csv')

In [ ]:
class AlmacenesSiModel:
    
    def __init__(self, serialized_models_path: str, campaigns_filepath: str, year_to_forecast: int, prior_year_sales_file_path: str):
        self.serialized_models_path = serialized_models_path
        self.year_to_forecast = year_to_forecast
        self.campaigns_filepath = campaigns_filepath
        
        self.future_regressors = pd.read_csv(self.campaigns_filepath)
        logging.info('Archivo de campañas cargado exitosamente ✅')
        self.prior_year_sales_df = pd.read_csv(prior_year_sales_file_path)
        logging.info(f'Archivo historico de ventas del año {self.year_to_forecast - 1} cargado exitosamente ✅')
        self.future_regressors['ds'] = pd.to_datetime(self.future_regressors['ds'])
        logging.info('Cargando datos del Modelo Predictivo ⌛')
        self.models_info = self.get_keys_names_and_model()
        logging.info('Modelo Predictivo cargado exitosamente ✅')
        
        
        
    @staticmethod
    def load_model_from_json(file_path):
        with open(file_path, "rb") as file:
            prophet_model = model_from_json(file.read())
        
        return prophet_model    
    
    @staticmethod
    def make_predictions(model, future_regressors : pd.DataFrame, year : int)->pd.DataFrame:
        # Obtener la última fecha histórica
        last_date = max(model.history_dates) + timedelta(weeks=1)
        
        # Crear un DataFrame con las fechas futuras hasta el año especificado, por semana comenzando en lunes
        future_dates = pd.date_range(start=last_date, end=f'{year}-12-31', freq='W-MON')
        
        # Crear el DataFrame con las fechas y los regresores
        future = pd.DataFrame({'ds': future_dates})
        
        # Añadir los regresores al DataFrame futuro
        future = future.merge(future_regressors, how='left', on='ds')
        
        # Llenar NaNs con 0 si es necesario 
        future.fillna(0, inplace=True)
        
        # Hacer la predicción
        forecast = model.predict(future)
        
        return forecast[['ds', 'yhat']]
    
    def get_keys_names_and_model(self):
        
        models_info = {}
        for model_path in tqdm(glob(f'{self.serialized_models_path}/*.json')):
            
            key_name = str(os.path.basename(model_path).split('.')[0].strip())
            model_temp = self.load_model_from_json(model_path)
            models_info[key_name] = model_temp
            
        return models_info
        
    def get_all_keys_prediction(self):
        
        forecast_df = pd.DataFrame()
        
        logging.info(f'Calculando Predicciones para el año {self.year_to_forecast} ⌛')
        for key, model in tqdm(self.models_info.items()):
            
            PARAMS = {
                'model' : model,
                'future_regressors' : self.future_regressors,
                'year' : self.year_to_forecast
            }
            try:
                forecast_temp = self.make_predictions(**PARAMS )
                forecast_temp['llave'] = key
                forecast_df = pd.concat([forecast_df,forecast_temp])
            except Exception as e:
                print(e)
                
        self.forecast_df = forecast_df
        self.forecast_df.rename(columns = {'ds' : 'fecha','yhat' : 'prediccion_demanda'}, inplace = True)
        return forecast_df
    
    
    def calculate_store_breakdown(self):
        
        forecast_df = self.forecast_df
        prior_year_sales_df = self.prior_year_sales_df
        forecast_df['fecha'] = pd.to_datetime(forecast_df['fecha'])
        forecast_df['semana'] = pd.to_datetime(forecast_df['fecha']).apply(lambda x: x.strftime('%-W'))

        prior_year_sales_df['date'] = pd.to_datetime(prior_year_sales_df['date'])
        prior_year_sales_df['week'] = prior_year_sales_df['date'].apply(lambda x: x.strftime('%-W'))
        
        self.demanda_desagrada_por_tienda = pd.DataFrame()
        year_prediction = int(forecast_df['fecha'].max().strftime('%Y'))
        for semana in forecast_df['semana'].unique(): # recorro todas las llaves disponibles en la prediccion
            logging.info(f'semana: {semana}')
            llaves_to_explore = forecast_df[forecast_df['semana'] == semana]['llave'].unique()# saco todas las llaves a explorar en la semana T
            df_week_selected =  prior_year_sales_df[(prior_year_sales_df['week'] == semana)] # saco la informacion del ultimo año disponible en la semana T
            # ------------------
            for llave in tqdm(llaves_to_explore): # recorro todas las llaves disponibles en la semana T

                demanda_forecast = forecast_df[(forecast_df['semana'] == semana) & (forecast_df['llave'] == llave)]['prediccion_demanda'] # calculo de la prediccion de la demanda para X llave
                demanda_forecast = np.ceil(demanda_forecast.iat[0]) # redondeo de la prediccion
                df_week_key_selected = df_week_selected[df_week_selected['combination'] == llave] # filtrado de la cantidad de unidades vendidad para la semana T para la llave X en el ultimo año de data
                if len(df_week_key_selected) > 0:
                    data_grouped_by_store = df_week_key_selected.groupby(['store_id'], as_index=False)['quantity'].sum() # calculo de cuantas unidades se vendieron por tienda en la semana T para la llave X
                    store_proportion = data_grouped_by_store.copy()
                    store_proportion['proportion'] = round(store_proportion['quantity'] / (store_proportion['quantity'].sum()),2) # calculo de porcentaje de las unidades vendidas por tienda, en la semana T para la llave X
                    store_proportion = store_proportion[store_proportion['proportion'] != 0] # remover aquellas tiendas que no vendieron ninguna unidad de la llave X para la semana T
                    store_proportion['forecast_llave'] = demanda_forecast
                    store_proportion['demanda'] = store_proportion['forecast_llave'] * store_proportion['proportion'] # multiplico la proporcion de ventas la llave X en la semana T para cada tienda, segun pronostico
                    store_proportion['demanda'] = store_proportion['demanda'].apply(lambda x: np.ceil(x)) # redondeo de la prediccion
                    temp_store_key_prediction = store_proportion.copy()
                    temp_store_key_prediction = temp_store_key_prediction[['store_id','demanda']]
                    temp_store_key_prediction['llave'] = llave
                    temp_store_key_prediction['week'] = semana
                    temp_store_key_prediction['year'] = year_prediction
                    self.demanda_desagrada_por_tienda = pd.concat([self.demanda_desagrada_por_tienda,temp_store_key_prediction])
                    
        return self.demanda_desagrada_por_tienda
                    
    

In [ ]:
PARAMS = {
    'serialized_models_path' : './serialized_models',
    'campaigns_filepath' : './archivos_insumo/archivo_insumo_dummy_campañas_2024.csv',
    'prior_year_sales_file_path' : './datasets/historico_ventas_2023_semanal.csv',
    'year_to_forecast' : 2024,
}
x = AlmacenesSiModel(**PARAMS)

In [ ]:
forecast = x.get_all_keys_prediction()

In [75]:
forecast['fecha'].min()

Timestamp('2019-01-07 00:00:00')

In [ ]:
demanda_desagrada_por_tienda = x.calculate_store_breakdown()

In [83]:
last_date = max(model.history_dates) + timedelta(weeks=1)
year = 2024
# Crear un DataFrame con las fechas futuras hasta el año especificado, por semana comenzando en lunes
future_dates = pd.date_range(start=last_date, end=f'{year}-12-31', freq='W-MON')

# Crear el DataFrame con las fechas y los regresores
future = pd.DataFrame({'ds': future_dates})

# Añadir los regresores al DataFrame futuro
archivos_insumo = pd.read_csv('./archivos_insumo/archivo_insumo_dummy_campañas_2024.csv')
archivos_insumo['ds'] = pd.to_datetime(archivos_insumo['ds'])
future = future.merge(archivos_insumo, how='left', on='ds')

# Llenar NaNs con 0 si es necesario 
future.fillna(0, inplace=True)

# Hacer la predicción
forecast = model.predict(future)

In [85]:
forecast.ds.min()

Timestamp('2024-01-01 00:00:00')